This script is to get the activation thresholds for various carrier frequencies and depths of modulation 

In [3]:
import numpy as np
import h5py
import matplotlib.pyplot as plt
import math
import os, sys
#Add main file to path
sys.path.insert(1,r'PATH WHERE PROCESSING_FUNCTIONS IS SAVED')

#import commonly used processing functions 
import processing_functions as process
from scipy.io import savemat

In [4]:
#carrier frequency selected files
Frequencies=[0, 500, 2000,5000,10000]

paths=[]
paths.append(r'PATH WHERE THE HDF5 FILES ARE SAVED')

for path in paths:
    Voltage_thresholds=[]
    Current_thresholds=[]
    Titration_thresholds=[]
    time=[]
    membrane_potential=[]

    mdic={}
    mdic['Frequencies']=Frequencies

    for F in Frequencies:
        print(F)
        open_this=os.path.join(path,(str(F) +".hdf5"))
        f=h5py.File(open_this,"r")
        
        #Get the thresholds
        Voltage_thresholds.append(f.attrs['Voltage threshold'])
        Current_thresholds.append(f.attrs['Current threshold']*1000)
        Titration_thresholds.append(f.attrs['Titration Voltage'])

        #Get the membrane potential of the main nerve
        folder=f.get('Nerve Membrane Potential')
        axon_name='Main'
        axon_folder=folder.get(axon_name)
        nodes=axon_folder.keys()
        node_numbers=[]
        axons_activation=[]
        axons_time=[]
        for node in nodes:
            node_number=process.get_node(node)
            node_numbers.append(node_number)
            axon_activation=axon_folder.get(node)
            axons_activation.append(axon_activation[1,:])
            axons_time=axon_activation[0,:]
        
        f.close()
        axon_activation_name='Axon_activation_'+str(F)
        mdic[axon_activation_name]=axons_activation
        node_numbers_name='Node_numbers_'+str(F)
        mdic[node_numbers_name]=node_numbers

    mdic['Axons_time']=axons_time
    mdic['Voltage_thresholds']=Voltage_thresholds
    mdic['Current_thresholds']=Current_thresholds
    mdic['Titration_thresholds']=Titration_thresholds
    file_name='Carrier_thresholds.mat'
    savemat(os.path.join(path,file_name),mdic)

0


In [ ]:
#Depth of Modulation selected files
Depths=[0, 25, 50, 75, 100]
paths=[]
paths.append(r'PATH WHERE THE HDF5 FILES ARE SAVED')

for path in paths:
    Voltage_thresholds=[]
    Current_thresholds=[]
    Titration_thresholds=[]
    time=[]
    membrane_potential=[]

    for depth in Depths:
        open_this=os.path.join(path,(str(depth) +".hdf5"))
        f=h5py.File(open_this,"r")
        
        #Get the thresholds
        Voltage_thresholds.append(f.attrs['Voltage threshold'])
        Current_thresholds.append(f.attrs['Current threshold']*1000)
        Titration_thresholds.append(f.attrs['Titration Voltage'])

        #Get the membrane potential of the main nerve
        folder=f.get('Nerve Membrane Potential')
        axon_name='Main'
        axon_folder=folder.get(axon_name)
        nodes=axon_folder.keys()
        last_node=0
        for node in nodes:
            node_number=process.get_node(node)
            if node_number>last_node:
                last_node=node_number
        node_before_last=str(last_node-1)
        node_before_last=str(1)
        saved_node=next((s for s in nodes if node_before_last in s), None)
        axon_activation=axon_folder.get(saved_node)
        
        

        f.close()

    mdic={'Depths':Depths,'Voltage_thresholds':Voltage_thresholds, 'Current_thresholds':Current_thresholds,'Titration_thresholds':Titration_thresholds}
    file_name='Depth_thresholds.mat'
    savemat(os.path.join(path,file_name),mdic)
    

In [4]:
#carrier frequency automated [This goes through all the participant files and generates the .mat files]
# This only works if you follow the convention of naming starting with P followed by the participant number but can, obviously, be adjusted to match your chosen naming convention

Frequencies=[0, 500, 2000,5000,10000]
Levels=['Sensory','Motor']
parent_path=r'PATH WHERE THE HDF5 FILES ARE SAVED'
Participants=list(range(4,13,1))  # Change as suitable
save_path=r'PATH WHERE YOU WANT THE MAT FILES TO BE EXPORTED'
for P in Participants:
	mdic={}
	for L in Levels:
		path=parent_path+'P'+str(P)+'/'+L+'/Frequency'
		Voltage_thresholds=[]
		Current_thresholds=[]
		Titration_thresholds=[]
		time=[]
		membrane_potential=[]

		for F in Frequencies:
			open_this=os.path.join(path,(str(F) +".hdf5"))
			f=h5py.File(open_this,"r")
			
			#Get the thresholds
			Voltage_thresholds.append(f.attrs['Voltage threshold'])
			Current_thresholds.append(f.attrs['Current threshold']*1000)
			Titration_thresholds.append(f.attrs['Titration Voltage'])

			#Get the membrane potential of the main nerve
			folder=f.get('Nerve Membrane Potential')
			axon_name='Main'
			axon_folder=folder.get(axon_name)
			nodes=axon_folder.keys()
			last_node=0
			for node in nodes:
				node_number=process.get_node(node)
				if node_number>last_node:
					last_node=node_number
			node_before_last=str(last_node-1)
			node_before_last=str(1)
			saved_node=next((s for s in nodes if node_before_last in s), None)
			axon_activation=axon_folder.get(saved_node)
			
			f.close()

		V_thresholds_tag=L+'_Voltage_thresholds'
		I_thresholds_tag=L+'_Current_thresholds'
		T_thresholds_tag=L+'_Titration_thresholds'
		mdic[V_thresholds_tag]=Voltage_thresholds
		mdic[I_thresholds_tag]=Current_thresholds
		mdic[T_thresholds_tag]=Titration_thresholds
		
	mdic['Frequencies']=Frequencies
	file_name='P'+str(P)+'_Carrier_thresholds.mat'
	savemat(os.path.join(save_path,file_name),mdic)

In [3]:
#Depth of Modulation automated [This goes through all the participant files and generates the .mat files]

Depths=[0, 25, 50, 75, 100]
Levels=['Sensory','Motor']
parent_path=r'PATH WHERE THE HDF5 FILES ARE SAVED'

Participants=list(range(4,13,1))
save_path=r'PATH WHERE YOU WANT THE MAT FILES TO BE EXPORTED'

for P in Participants:
	mdic={}
	for L in Levels:
		path=parent_path+'P'+str(P)+'/'+L+'/Depth of Modulation'
		Voltage_thresholds=[]
		Current_thresholds=[]
		Titration_thresholds=[]
		time=[]
		membrane_potential=[]

		for depth in Depths:
			open_this=os.path.join(path,(str(depth) +".hdf5"))
			f=h5py.File(open_this,"r")
			
			#Get the thresholds
			Voltage_thresholds.append(f.attrs['Voltage threshold'])
			Current_thresholds.append(f.attrs['Current threshold']*1000)
			Titration_thresholds.append(f.attrs['Titration Voltage'])

			#Get the membrane potential of the main nerve
			folder=f.get('Nerve Membrane Potential')
			axon_name='Main'
			axon_folder=folder.get(axon_name)
			nodes=axon_folder.keys()
			last_node=0
			for node in nodes:
				node_number=process.get_node(node)
				if node_number>last_node:
					last_node=node_number
			node_before_last=str(last_node-1)
			node_before_last=str(1)
			saved_node=next((s for s in nodes if node_before_last in s), None)
			axon_activation=axon_folder.get(saved_node)
			
			f.close()

		V_thresholds_tag=L+'_Voltage_thresholds'
		I_thresholds_tag=L+'_Current_thresholds'
		T_thresholds_tag=L+'_Titration_thresholds'
		mdic[V_thresholds_tag]=Voltage_thresholds
		mdic[I_thresholds_tag]=Current_thresholds
		mdic[T_thresholds_tag]=Titration_thresholds
		
	mdic['Depths']=Depths
	file_name='P'+str(P)+'_Depth_thresholds.mat'
	savemat(os.path.join(save_path,file_name),mdic)

In [9]:
f.close()